In [1]:
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
from shapely import wkt
from cartoframes.viz import *
import networkx as nx

In [2]:
# 데이터 로드
dong_boundary = gpd.read_file('../emd2/행정구역.shp', encoding='CP949')
dong = dong_boundary[dong_boundary['ADM_CD'].str.startswith('11')]

dong['ADM_CD'] = dong['ADM_CD'].astype(str)

# 중복이름 동 방지
dong.loc[(dong['ADM_NM']=='신사동') & (dong['ADM_CD']=='1121068'), 'ADM_NM'] = '신사동(강남)'
dong.loc[(dong['ADM_NM']=='신사동') & (dong['ADM_CD']=='1123051'), 'ADM_NM'] = '신사동(관악)'

# CRS 변환
dong.set_crs(epsg=5179, inplace=True)
dong.to_crs(epsg=4326, inplace=True)

# 결과 확인
print(dong.crs)
print(dong.shape)

EPSG:4326
(424, 7)


In [3]:
print(dong.shape)

(424, 7)


In [4]:
# layer_dong = Layer(dong,
#                     basic_style(opacity=0.2), # 투명도
#                     popup_hover=popup_element('ADM_NM')) # 얹으면 팝업

# layer_dong

# 동코드 확인

In [4]:
dong_mapping = pd.read_excel('../Dataset/dong_mapping.xlsx')
dong_mapping.columns = dong_mapping.iloc[0]
dong_mapping['H_SDNG_CD'] = dong_mapping['H_SDNG_CD'].astype(str)
dong_mapping = dong_mapping.iloc[1:].reset_index(drop=True)
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1121068', 'H_DNG_NM'] = '신사동(관악)'
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1123051', 'H_DNG_NM'] = '신사동(강남)'
dong_mapping    # H_SDNG_CD: 통계청코드,	H_DNG_CD: 행안부코드

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
0,1101053,11110530,서울,종로구,사직동
1,1101054,11110540,서울,종로구,삼청동
2,1101055,11110550,서울,종로구,부암동
3,1101056,11110560,서울,종로구,평창동
4,1101057,11110570,서울,종로구,무악동
...,...,...,...,...,...
419,1125070,11740690,서울,강동구,둔촌1동
420,1125071,11740700,서울,강동구,둔촌2동
421,1125072,11740570,서울,강동구,암사1동
422,1125073,11740610,서울,강동구,천호2동


In [5]:
# dong_mapping 기준으로 맞추기
map_dict = {'종로1.2.3.4가동': '종로1·2·3·4가동',
            '종로5.6가동': '종로5·6가동',
            '금호2.3가동': '금호2·3가동',
            '면목3.8동': '면목3·8동',
            '상계3.4동': '상계3·4동',
            '상계6.7동': '상계6·7동',
            '중계2.3동': '중계2·3동'
            }

# 이름 일치 시키기
for key, val in map_dict.items():
    dong_mapping.loc[dong_mapping['H_DNG_NM']==key, 'H_DNG_NM'] = val

# 일원2동 -> dong에 없음
# 상일동 -> 상일1동, 상일2동
# 항동 -> dongmapping에 없음
# 개포3동 -> dongmapping에 없음

code2name_dict = dong_mapping.set_index('H_DNG_CD')['H_DNG_NM'].to_dict()
#code_name_dict

In [6]:
dm_i = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='inner')
dm_o = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='outer')
dm_l = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='left')
dm_r = pd.merge(dong_mapping, dong, left_on='H_DNG_NM', right_on='ADM_NM', how='right')

print(f"inner: {len(dm_i)}")
print(f"outer: {len(dm_o)}")
print(f"left: {len(dm_l)}")
print(f"right: {len(dm_r)}")

inner: 424
outer: 424
left: 424
right: 424


In [7]:
not_in_left = list(dm_l.loc[dm_l['ADM_NM'].isna(), 'H_DNG_NM'])
display(dm_l.loc[dm_l['ADM_NM'].isna()])

not_in_right = list(dm_r.loc[dm_r['H_DNG_NM'].isna(), 'ADM_NM'])
display(dm_r.loc[dm_r['H_DNG_NM'].isna()])

# 일원2동 -> 
# 항동 -> 오류2동 값 반으로 나누기
# 개포3동 -> 개포2동의 값을 반으로 나누기
# 상일1, 2동 -> 상일동의 값을 반으로 나누기

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM,SIDO_CD,SIDO_NM,SIGUNGU_CD,SIGUNGU_NM,ADM_CD,ADM_NM,geometry


,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM,SIDO_CD,SIDO_NM,SIGUNGU_CD,SIGUNGU_NM,ADM_CD,ADM_NM,geometry


In [8]:
dong[dong['ADM_NM'].str.contains('개포')]

,SIDO_CD,SIDO_NM,SIGUNGU_CD,SIGUNGU_NM,ADM_CD,ADM_NM,geometry
368,11,서울특별시,11230,강남구,1123068,개포1동,"POLYGON ((127.05826 37.48474, 127.05924 37.483..."
369,11,서울특별시,11230,강남구,1123071,개포4동,"POLYGON ((127.05176 37.48479, 127.05199 37.484..."
378,11,서울특별시,11230,강남구,1123080,개포2동,"POLYGON ((127.07277 37.49191, 127.07290 37.491..."


In [9]:
dong[dong['ADM_NM'].str.contains('일원')]

,SIDO_CD,SIDO_NM,SIGUNGU_CD,SIGUNGU_NM,ADM_CD,ADM_NM,geometry
370,11,서울특별시,11230,강남구,1123072,일원본동,"POLYGON ((127.09012 37.49099, 127.09012 37.490..."
371,11,서울특별시,11230,강남구,1123073,일원1동,"POLYGON ((127.09949 37.49284, 127.09950 37.492..."
372,11,서울특별시,11230,강남구,1123074,일원2동,"POLYGON ((127.08338 37.49994, 127.08348 37.499..."


In [10]:
dong_mapping[dong_mapping['H_DNG_NM'].str.contains('상일')]

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
407,1125052,11740520,서울,강동구,상일동


# Pablo Data (Human Flow)

In [11]:
dong_mapping = pd.read_excel('../Dataset/dong_mapping.xlsx')
dong_mapping.columns = dong_mapping.iloc[0]
dong_mapping['H_SDNG_CD'] = dong_mapping['H_SDNG_CD'].astype(str)
dong_mapping = dong_mapping.iloc[1:].reset_index(drop=True)
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1121068', 'H_DNG_NM'] = '신사동(관악)'
dong_mapping.loc[dong_mapping['H_SDNG_CD']=='1123051', 'H_DNG_NM'] = '신사동(강남)'
dong_mapping    # H_SDNG_CD: 통계청코드,	H_DNG_CD: 행안부코드

,H_SDNG_CD,H_DNG_CD,DO_NM,CT_NM,H_DNG_NM
0,1101053,11110530,서울,종로구,사직동
1,1101054,11110540,서울,종로구,삼청동
2,1101055,11110550,서울,종로구,부암동
3,1101056,11110560,서울,종로구,평창동
4,1101057,11110570,서울,종로구,무악동
...,...,...,...,...,...
419,1125070,11740690,서울,강동구,둔촌1동
420,1125071,11740700,서울,강동구,둔촌2동
421,1125072,11740570,서울,강동구,암사1동
422,1125073,11740610,서울,강동구,천호2동


In [12]:
# dong_mapping 기준으로 맞추기
map_dict = {'종로1.2.3.4가동': '종로1·2·3·4가동',
            '종로5.6가동': '종로5·6가동',
            '금호2.3가동': '금호2·3가동',
            '면목3.8동': '면목3·8동',
            '상계3.4동': '상계3·4동',
            '상계6.7동': '상계6·7동',
            '중계2.3동': '중계2·3동'
            }

# 이름 일치 시키기
for key, val in map_dict.items():
    dong_mapping.loc[dong_mapping['H_DNG_NM']==key, 'H_DNG_NM'] = val

# 일원2동 -> dong에 없음
# 상일동 -> 상일1동, 상일2동
# 항동 -> dongmapping에 없음
# 개포3동 -> dongmapping에 없음

code2name_dict = dong_mapping.set_index('H_DNG_CD')['H_DNG_NM'].to_dict()
#code_name_dict

In [13]:
# 파블로 데이터 수정
hf_df = pd.read_csv('../Dataset/PabloTask_AggregatedDataframe.csv')
hf_df['행정동코드'] = (hf_df['행정동코드'].astype(int)).astype(str)

dong_codes = list(hf_df['행정동코드'].unique())

# 동이름으로 변경
for i in range(len(dong_codes)):
    # 사전에 있는 이름일때
    if dong_codes[i] in code2name_dict:
        hf_df.loc[hf_df['행정동코드']==dong_codes[i], '행정동'] = code2name_dict[dong_codes[i]]   # 동이름으로 변경
    else:
        print(dong_codes[i])

In [14]:
hf_df

,기준일ID,행정동코드,총생활인구수,남자0세부터9세생활인구수,남자10세부터14세생활인구수,남자15세부터19세생활인구수,남자20세부터24세생활인구수,남자25세부터29세생활인구수,남자30세부터34세생활인구수,남자35세부터39세생활인구수,...,여자60세부터64세생활인구수,여자65세부터69세생활인구수,여자70세이상생활인구수,총생활인구수_LONG_FOREIGNER,중국인체류인구수_LONG_FOREIGNER,중국외외국인체류인구수_LONG_FOREIGNER,총생활인구수_TEMP_FOREIGNER,중국인체류인구수_TEMP_FOREIGNER,중국외외국인체류인구수_TEMP_FOREIGNER,행정동
0,201701,11110515,15948.599397,0.033578,0.021396,0.022550,0.041458,0.043958,0.040945,0.045796,...,0.027329,0.023583,0.076797,400.106945,0.446555,0.553445,839.632493,0.542232,0.457769,청운효자동
1,201701,11110530,32795.242545,0.017176,0.007634,0.011646,0.033534,0.042079,0.049523,0.056386,...,0.030691,0.021816,0.077705,1536.160386,0.438137,0.561863,2162.551359,0.417027,0.582973,사직동
2,201701,11110540,6013.897520,0.012980,0.009946,0.032842,0.041308,0.054209,0.048953,0.050377,...,0.029532,0.024999,0.079635,314.622914,0.422761,0.577239,626.893084,0.449886,0.550116,삼청동
3,201701,11110550,11578.176069,0.022130,0.017159,0.026907,0.042039,0.031760,0.024583,0.030687,...,0.041363,0.030520,0.104066,358.957612,0.456152,0.543848,286.697474,0.291895,0.708108,부암동
4,201701,11110560,22895.411724,0.027763,0.020082,0.025157,0.031510,0.025448,0.022607,0.027102,...,0.044217,0.032941,0.113925,460.723224,0.394245,0.605755,211.288792,0.177801,0.822201,평창동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28403,202207,11140635,14530.991262,0.040389,0.014614,0.015383,0.021771,0.029629,0.032346,0.042020,...,0.040907,0.030339,0.099378,384.796756,0.393653,0.606347,133.851405,0.211473,0.786151,약수동
28404,202207,11140590,23176.182660,0.010805,0.005220,0.015364,0.040066,0.048162,0.049410,0.059865,...,0.025694,0.018772,0.040036,2481.416539,0.522951,0.477049,2541.894839,0.251054,0.748739,광희동
28405,202207,11230740,22796.005607,0.018643,0.011895,0.034970,0.092788,0.068043,0.035076,0.029976,...,0.021436,0.017203,0.053177,4484.964256,0.709483,0.290517,298.606432,0.740030,0.257845,이문1동
28406,202207,11290760,20724.229083,0.038519,0.026098,0.027060,0.029620,0.027357,0.029521,0.036693,...,0.040665,0.032259,0.073779,310.405583,0.482099,0.517901,41.503501,0.273598,0.723447,장위1동


In [15]:
hf_df_dongs = list(hf_df['행정동'].unique())
len(hf_df_dongs)

424

# 코사인 유사도 행렬 불러오기

In [1]:
c_hf = pd.read_csv('../Pablo/results/cosineSimilarity_201701.csv')
cols = list(c_hf.columns)

for i in range(len(cols)):
    cols[i] = code2name_dict[cols[i][:-2]]

c_hf.columns = cols
c_hf

NameError: name 'pd' is not defined

In [2]:
mean_similarity = c_hf.values[np.triu_indices_from(c_hf.values, k=1)].mean()              # mean
third_quartile = np.quantile(c_hf.values[np.triu_indices_from(c_hf.values, k=1)], 0.75)   # 3Q

print(mean_similarity)
print(third_quartile)

# make a graph
G = nx.Graph()
# add nodes

nodes = c_hf.columns.tolist()  # Add each dong as a node.

G.add_nodes_from(nodes)

# add edge: Create an edge if the similarity is greater than the third quartile.
for i, node1 in enumerate(nodes):
    for j, node2 in enumerate(nodes):
        if i < j:     # Search only the upper triangular matrix
            similarity = c_hf.iloc[i, j]
            if similarity > third_quartile:
                G.add_edge(node1, node2, weight=similarity)
# check graph
print("Nodes:", len(G.nodes()))
print("Edges:", G.edges(data=True))

NameError: name 'c_hf' is not defined